# VectorStoreRetrieverMemory
## 概覽

```VectorStoreRetrieverMemory``` 將記憶儲存在向量資料庫中，每次被調用時會查詢排名前 K 的「相關」文件。  
這與其他大多數記憶類別不同，因為它不會明確追蹤對話的時間順序。

在本教學中，我們將透過模擬面試場景，探索 ```VectorStoreRetrieverMemory``` 的實務應用。透過此範例，我們將看到 ```VectorStoreRetrieverMemory``` 是如何根據語意相關性來搜尋資訊，而非依照對話的時間順序。

### 目錄

- [概覽](#overview)
- [環境設置](#environment-setup)
- [初始化向量資料庫](#initializing-the-vector-store)
- [儲存面試對話](#saving-interview-conversations)
- [檢索相關對話](#retrieving-relevant-conversations)

### 參考資料

- [LangChain Python API 參考 > langchain: 0.3.13 > memory > VectorStoreRetrieverMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.vectorstore.VectorStoreRetrieverMemory.html)
- [Faiss](https://github.com/facebookresearch/faiss)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    ["langchain_community", "langchain_openai", "langchain_core", "faiss-cpu"],
    verbose=False,
    upgrade=False,
)

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "VectorStoreRetrieverMemory",
    }
)

Environment variables have been set successfully.


You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it.

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [4]:
from dotenv import load_dotenv

# Load API key information
load_dotenv(override=True)

True

## Initializing the Vector Store

Next, we'll set up our vector store using FAISS. FAISS is an efficient similarity search library that will help us store and retrieve conversation embeddings:

In [5]:
import faiss
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores.faiss import FAISS

# Initialize OpenAI embeddings
embeddings_model = OpenAIEmbeddings()

# Create FAISS index
embedding_size = 1536  # Size for OpenAI embeddings
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})

This setup creates an in-memory vector store that will maintain our conversation embeddings for quick similarity searches.

## 儲存面試對話

接下來，我們將建立記憶系統，並填入範例面試對話。

請注意，設定 ```k=1``` 可確保只回傳最相關的一則對話。（在實際應用中，你可能會希望增加此數值，以提供更多上下文。）

In [ ]:
from langchain.memory import VectorStoreRetrieverMemory

# We set k=1 to show that vector lookups still can return semantically relevant information
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
memory = VectorStoreRetrieverMemory(retriever=retriever)
#請winsruf幫我把所有的md英文或是英文的prompt都改成中文

# Save arbitrary conversations
memory.save_context(
    inputs={
        "human": "哈囉, 謝謝你來面試, 請介紹自己"
    },
    outputs={
        "ai": "你好, 我是位專注於開發的開發者, 我主修計算機科學, 在大學時主要使用 Java 和 Python, 最近我參與了一個 Web 開發專案, 在這個專案中我學習了如何開發服務以供實際使用者使用。"
    },
)
memory.save_context(
    inputs={"human": "你的角色在專案中是什麼?"},
    outputs={
        "ai": "我的角色是後端開發者, 我負責處理使用者資料, 開發服務器邏輯, 實現 RESTful API 以供前端使用, 我也參與了資料庫設計"
    },
)
memory.save_context(
    inputs={
        "human": "你有什麼困難在小組專案中, 你是如何解決的?"
    },
    outputs={
        "ai": "我們在專案初期有些溝通問題。為了解決這個問題, 我們的團隊定期舉行會議, 分享每個成員的進度。此外, 當問題出現時, 我們積極分享意見並共同努力找到合理的解決方案。"
    },
)
memory.save_context(
    inputs={"human": "你認為自己作為開發者的強項是什麼?"},
    outputs={
        "ai": "我的強項是快速學習能力和問題解決能力。我能夠快速掌握新的技術和工具，當面臨複雜問題時，我能夠提出創意解決方案。此外，我非常重視團隊合作，認為與同事合作很重要。"
    },
)

/var/folders/b5/3pdyzl995m5_37d6rr9vfbh00000gn/T/ipykernel_31365/1922969020.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = VectorStoreRetrieverMemory(retriever=retriever)


## Retrieving Relevant Conversations

讓我們看看系統如何根據查詢检索相關信息:

In [ ]:
# Query about educational background
print("Query: hat was the interviewee's major?")
print(
    memory.load_memory_variables({"prompt": "What was the interviewee's major?"})[
        "history"
    ]
)

Query: What was the interviewee's major?
human: Hello, thank you for coming to the interview today. Please introduce yourself.
ai: Hello. I'm a junior developer who majored in Computer Science. In college, I mainly used Java and Python, and recently, I participated in a web development project where I gained experience developing services for real users.


In [8]:
# Query about project experience
print("Query: What was the interviewee's role in the project?")
print(
    memory.load_memory_variables(
        {"human": "What was the interviewee's role in the project?"}
    )["history"]
)

Query: What was the interviewee's role in the project?
human: What was your role in the project?
ai: My role was as a backend developer. I was responsible for processing user data and developing server logic, implementing RESTful APIs for communication with the frontend. I also participated in database design.


這種方法在建構需要根據過去對話上下文進行參考的系統時特別有價值，例如客服機器人、教育助理，或任何需要維持具上下文意識對話歷史的應用場景。